In [71]:
import openai
import textwrap

In [17]:
# openai.Model.list()

In [4]:
with open('api_key.txt', 'r') as f:
    openai.api_key = f.read().strip()

In [14]:
# # Make a request with ada
# response = openai.Completion.create(
#     engine='ada',
#     prompt='This is a test',
#     temperature=0.6,
#     max_tokens=5,
#     top_p=1)

In [19]:
GPT_MODEL = 'gpt-3.5-turbo'

In [ ]:
openai.ChatCompletion.create(
  model=GPT_MODEL,
  messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': 'Who won the world series in 2020?'},
        {'role': 'assistant', 'content': 'The Los Angeles Dodgers won the World Series in 2020.'},
        {'role': 'user', 'content': 'Where was it played?'}
    ]
)

In [66]:
"""
Generating research ideas
1. Zero shot ask for ideas
2. Feed it papers, and ask for ideas

Generate a plan for a project
""";

In [82]:
IDEA_SYSTEM_PROMPT = \
'You are GPT-Research and your purpose is to generate high quality, ML research ideas. ' + \
'You should generate ideas based on the user\'s areas of preference. ' + \
'If the ideas fall into multiple areas, that is ideal, but not necessary. ' + \
'Research ideas should fit the following criteria:\n' + \
'1. The idea should be novel and focused around understanding something new\n' + \
'2. Proof of concept experiments should be relatively simple to implement, ' + \
'not requiring exessive compute or large codebases\n' + \
'3. Be very specific and not too general. Niche is good.\n' + \
'4. Not require external tools/data to test.\n' + \
'5. Be focused on fundemental issues and understanding.\n' + \
'Ideas should be listed as I1. {IDEA 1}\nI2. {IDEA 2}, etc. ' + \
'Start directly with "I1.", no fluff.'

PROJECT_SYSTEM_PROMPT = \
'You are GPT-Research and you generate research project outlines from ideas. ' + \
'The project outline should specify the different things that need to be coded ' + \
'and the experiments that need to be run to get to the point where the idea ' + \
'could be evaluated and potentially published. ' + \
'The project outline should be specific enough that a new ' + \
'grad student could implement it. It should be very detailed. ' + \
'The steps should be formatted as: S1. {STEP 1}\nS2. {STEP 2}, etc.\n' + \
'If some parts of the project will be dependent on other parts, ' + \
'only include the steps that are independent, then add a note ' + \
'at the end of the response in the format: "FOLLOW UP WORK: ' + \
'{HIGH-LEVEL EXPLANATION OF FUTURE DIRECTION BASED ON RESULTS}. ' + \
'Start directly with "S1.", no fluff.'

In [83]:
def generate_ideas(areas_of_interest, n=5, details=None, history=None):
  """
  areas_of_interest: list of strings
  """
  user_prompt = f'Generate {n} research ideas for the following areas of interest: ' + \
    ', '.join(areas_of_interest)
  if details is not None:
    user_prompt += f'. Details: {details}'
  
  if history is None:
    history = [
        {'role': 'system', 'content': IDEA_SYSTEM_PROMPT},
        {'role': 'user', 'content': user_prompt}
      ]
    
  response = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages=history,
    max_tokens=150,
    stop='I2.'
  )

  response = response['choices'][0]['message'].to_dict()
  history.append(response)
  content = response['content']

  return content, history

def generate_outline(idea, details=None, history=None):
  """
  idea: string
  """
  user_prompt = f'Generate a project outline for the following idea:\n{idea}'
  if details is not None:
    user_prompt += f'. Extra Details: {details}'
  
  if history is None:
    history = [
        # {'role': 'system', 'content': PROJECT_SYSTEM_PROMPT},
        {'role': 'user', 'content': PROJECT_SYSTEM_PROMPT},
        {'role': 'user', 'content': user_prompt}
      ]
    
  response = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages=history,
    max_tokens=400,
    stop='S3.'
  )

  response = response['choices'][0]['message'].to_dict()
  history.append(response)
  content = response['content']

  return content, history

In [79]:
areas_of_interest = ['RL', 'model-based RL', 'representation learning', 'continual learning']
details = None # 'I want ideas that focus on solving or understanding a problem'

response, hist = generate_ideas(
  areas_of_interest,
  details=details)

In [61]:
# Print response with text wrapping
print(textwrap.fill(response, 80))

I1. "Exploration in model-based RL with predictive models": Develop a new
exploration algorithm for model-based RL by leveraging predictive models to
improve the uncertainty estimates. This could be done by using variational
methods to learn the approximate posterior over latent representations that are
used to generate rollouts for the value and policy functions. Simple POC
experiments could be done using classic RL environments like Atari games or
robotic control tasks.  I


In [63]:
idea_idx = 1
chosen_idea = response.split(f'I{idea_idx}. ')[1].split('I')[0]

In [64]:
chosen_idea

'"Exploration in model-based RL with predictive models": Develop a new exploration algorithm for model-based RL by leveraging predictive models to improve the uncertainty estimates. This could be done by using variational methods to learn the approximate posterior over latent representations that are used to generate rollouts for the value and policy functions. Simple POC experiments could be done using classic RL environments like Atari games or robotic control tasks.\n\n'

In [84]:
details=None

outline_response, outline_history = generate_outline(chosen_idea, details=details)

In [85]:
print(textwrap.fill(outline_response, 80))

  S1. Develop a predictive model that can generate rollouts for a value and
policy function.  S2. Implement variational methods to learn the approximate
posterior over latent representations.
